In [1]:
from sklearn.datasets import make_classification, make_multilabel_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from collections import deque 
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import plotly.express as px

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [6]:
X,y = make_classification(15_000,500,random_state=50,n_classes=2, weights=[0.95])
X, X_validate, y, y_validate = train_test_split(X,y, random_state=50)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =  0.7)

In [9]:
# X_train_ = X_train.copy()
# y_train_ = y_train.copy()
# rng_idx = pd.Series(y_train).sample(len(y_train)).index
# X_train_ = X_train_[rng_idx]
# y_train_ = y_train_[rng_idx]
# M_idx, m_idx = pd.DataFrame(y_train_).groupby(0).apply(lambda x: x.index.values)

In [9]:
# pd.Series(y_train).value_counts()   

In [10]:
# bal_idx = pd.DataFrame(y_train, columns = ['y']).groupby('y').apply(lambda x: x.sample(250)).index.to_frame()[1].values


In [11]:
# rng = np.random.default_rng(1235)

In [12]:
# rng.shuffle(bal_idx)

In [13]:
# X_bal = X_train[bal_idx]
# y_bal = y_train[bal_idx]

In [14]:
# X_bal = X_train.copy()
# y_bal = y_train.copy()

In [15]:
# M_idx, m_idx = pd.DataFrame(y_bal).groupby(0).apply(lambda x: x.index.values)

---

In [20]:
 actions = {0 : 0, 1 : 1}
 len(actions)

2

In [63]:
%run -i ./DQN_Class.py
gamma=0.5
epsilon=.8
numberEpisodes= 120
LearningQDeep=DeepQLearning(gamma,epsilon,numberEpisodes)


# NN approach

In [64]:

nn = LearningQDeep.buildNetwork()

In [65]:
nn.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_120 (Dense)           (None, 30)                15030     
                                                                 
 dense_121 (Dense)           (None, 64)                1984      
                                                                 
 dense_122 (Dense)           (None, 20)                1300      
                                                                 
 dense_123 (Dense)           (None, 1)                 21        
                                                                 
Total params: 18,335
Trainable params: 18,335
Non-trainable params: 0
_________________________________________________________________


In [66]:

early_stopping = tf.keras.callbacks.EarlyStopping(
                                monitor='val_f1', 
                                verbose=1,
                                patience=100,
                                mode='max',
                                restore_best_weights=True)

In [67]:
nn.fit(X_train,y_train, epochs=1, callbacks = [early_stopping], batch_size=X_train.shape[0])

(3375,)
(3375,)
[0.104454264 0.0466095544 0.161626264 ... 0.113156877 0.217940584 0.0561327226]
1/1 [==============================] - 1s 899ms/step - loss: 0.1834 - recall_m: 0.0772 - precision_m: 0.0897 - f1: 0.0829


In [75]:
y_train[0]

1

In [79]:
np.array(y_train[0])

array(1)

In [84]:
np.array([y_train[0]])

array([1])

In [98]:
np.array(y_train[0:2])

array([1, 0])

In [107]:
nn.evaluate(X_train[0:3,:],np.array(y_train[0:3]))

[0.476387918 0.145877674 0.244506896]
1/1 [==============================] - 0s 43ms/step - loss: 0.2889 - recall_m: 0.0000e+00 - precision_m: 0.0000e+00 - f1: 0.0000e+00


[0.2889241576194763, 0.0, 0.0, 0.0]

In [109]:
np.array([0.476387918, 0.145877674, 0.244506896]).sum()/3

0.28892416266666665

In [103]:
(0.3819394)**2

0.14587770527236

In [101]:
nn.predict(X_train[1,:].reshape(1,-1))

array([[0.3819394]], dtype=float32)

In [72]:
y_train

array([1, 0, 0, ..., 0, 0, 0])

In [272]:
y_pred = [1 if x>0.5 else 0 for x in nn.predict(X_train)]

106/106 [==============================] - 0s 2ms/step


In [273]:
print(confusion_matrix(y_train, y_pred))

[[3056    0]
 [   0  319]]


In [274]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3056
           1       1.00      1.00      1.00       319

    accuracy                           1.00      3375
   macro avg       1.00      1.00      1.00      3375
weighted avg       1.00      1.00      1.00      3375



In [275]:
y_pred = [1 if x>0.5 else 0 for x in nn.predict(X_test)]

247/247 [==============================] - 0s 2ms/step


In [276]:
print(confusion_matrix(y_test, y_pred))

[[6795  225]
 [ 438  417]]


In [277]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      7020
           1       0.65      0.49      0.56       855

    accuracy                           0.92      7875
   macro avg       0.79      0.73      0.76      7875
weighted avg       0.91      0.92      0.91      7875



In [278]:
y_pred = [1 if x>0.5 else 0 for x in nn.predict(X_validate)]

118/118 [==============================] - 0s 1ms/step


In [279]:
print(confusion_matrix(y_validate, y_pred))

[[3255  118]
 [ 191  186]]


In [280]:
print(classification_report(y_validate, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      3373
           1       0.61      0.49      0.55       377

    accuracy                           0.92      3750
   macro avg       0.78      0.73      0.75      3750
weighted avg       0.91      0.92      0.91      3750



---

In [11]:
 actions = {0 : 0, 1 : 1}
 len(actions)

2

In [12]:
def reward(si, a, M, m, y):
    l = y[si]

    if (a == l) & (si in m):
        r = 1
    elif  (a != l) & (si in m):
        r = -1
    if (a == l) & (si in M):
        r = 0.04
    elif  (a != l) & (si in M):
        r = -0.04
    return r

In [13]:
def step(action, i, M, m, X, y):
    l = y[i]

    r = reward(i,action, M, m, y)

    if i == len(y) - 1:
        terminated = True
        return r, None, terminated, i+1

    nextState = X[i+1,:]

    if (action != l) & (i in m):
        # print(action, l)
        terminated = True
        return r, nextState, terminated, i+1

    terminated = False
    
    return r, nextState, terminated, i+1
    

In [15]:
%run -i ./DQN_Class.py
gamma=0.25
epsilon=.9
numberEpisodes= 500
LearningQDeep=DeepQLearning(gamma,epsilon,numberEpisodes)


In [16]:
0.96**(500-25)*(0.9)

3.4125435562551007e-09

In [17]:
LearningQDeep.mainNetwork.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               250500    
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                                 
 dense_2 (Dense)             (None, 2)                 1002      
                                                                 
Total params: 502,002
Trainable params: 502,002
Non-trainable params: 0
_________________________________________________________________


In [18]:

%%time
LearningQDeep.trainingEpisodes()

KeyboardInterrupt: 

In [19]:
loaded_model = LearningQDeep.mainNetwork

In [20]:
y_pred  = np.argmax(loaded_model.predict(X_train,verbose=0), axis = 1)

In [21]:

print(confusion_matrix(y_train, y_pred))

[[3109   85]
 [   1  180]]


In [22]:
print(classification_report(y_train,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.97      0.99      3194
           1       0.68      0.99      0.81       181

    accuracy                           0.97      3375
   macro avg       0.84      0.98      0.90      3375
weighted avg       0.98      0.97      0.98      3375



In [23]:
y_pred  = np.argmax(loaded_model.predict(X_test,verbose=0), axis = 1)

In [24]:
print(confusion_matrix(y_test, y_pred))

[[6795  639]
 [ 161  280]]


In [25]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.91      0.94      7434
           1       0.30      0.63      0.41       441

    accuracy                           0.90      7875
   macro avg       0.64      0.77      0.68      7875
weighted avg       0.94      0.90      0.91      7875



In [26]:
y_pred  = np.argmax(loaded_model.predict(X_validate,verbose=0), axis = 1)

In [27]:
print(confusion_matrix(y_validate, y_pred))

[[3235  324]
 [  73  118]]


In [28]:
print(classification_report(y_validate,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.91      0.94      3559
           1       0.27      0.62      0.37       191

    accuracy                           0.89      3750
   macro avg       0.62      0.76      0.66      3750
weighted avg       0.94      0.89      0.91      3750



In [29]:
LearningQDeep.mainNetwork.save('Trained_model.h5')

In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
rf = RandomForestClassifier()

In [32]:
rf.fit(X_train,y_train)

RandomForestClassifier()

In [33]:
y_pred  = rf.predict(X_train)

In [34]:

print(confusion_matrix(y_train, y_pred))

[[3194    0]
 [   0  181]]


In [35]:
print(classification_report(y_train,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3194
           1       1.00      1.00      1.00       181

    accuracy                           1.00      3375
   macro avg       1.00      1.00      1.00      3375
weighted avg       1.00      1.00      1.00      3375



In [36]:
y_pred  = rf.predict(X_test)

In [37]:
print(confusion_matrix(y_test, y_pred))

[[7413   21]
 [ 131  310]]


In [38]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7434
           1       0.94      0.70      0.80       441

    accuracy                           0.98      7875
   macro avg       0.96      0.85      0.90      7875
weighted avg       0.98      0.98      0.98      7875



In [39]:
y_pred  = rf.predict(X_validate)

In [40]:
print(confusion_matrix(y_validate, y_pred))

[[3551    8]
 [  69  122]]


In [41]:
print(classification_report(y_validate,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3559
           1       0.94      0.64      0.76       191

    accuracy                           0.98      3750
   macro avg       0.96      0.82      0.87      3750
weighted avg       0.98      0.98      0.98      3750

